In [19]:
import pandas as pd

df = pd.read_excel('./Data/drone_data.xlsx')

print(df.head())

   Maximum Principal Elastic Strain (m/m)  Total Deformation (m)  \
0                            1.645777e-08           1.194049e-09   
1                            1.462148e-08           1.061343e-09   
2                            1.278519e-08           9.286820e-10   
3                            1.096267e-08           7.960099e-10   
4                            9.174577e-09           6.633378e-10   

   Equivalent Elastic Strain (m/m) Dimensions (mm)  
0                     2.171072e-08     400*410*100  
1                     1.929829e-08     400*410*100  
2                     1.688586e-08     400*410*100  
3                     1.447343e-08     400*410*100  
4                     1.206215e-08     400*410*100  


In [20]:
# Split the 'dimensions' column into three separate columns
df[['length', 'width', 'height']] = df['Dimensions (mm)'].str.split('*', expand=True)

# Convert the new columns to numeric values
df['length'] = pd.to_numeric(df['length'])
df['width'] = pd.to_numeric(df['width'])
df['height'] = pd.to_numeric(df['height'])

df.drop('Dimensions (mm)', axis=1, inplace=True)

df

,Maximum Principal Elastic Strain (m/m),Total Deformation (m),Equivalent Elastic Strain (m/m),length,width,height
0,1.645777e-08,1.194049e-09,2.171072e-08,400,410,100
1,1.462148e-08,1.061343e-09,1.929829e-08,400,410,100
2,1.278519e-08,9.286820e-10,1.688586e-08,400,410,100
3,1.096267e-08,7.960099e-10,1.447343e-08,400,410,100
4,9.174577e-09,6.633378e-10,1.206215e-08,400,410,100
5,7.294672e-09,5.306771e-10,9.649259e-09,400,410,100
6,5.476743e-09,3.980049e-10,7.236944e-09,400,410,100
7,3.647336e-09,2.653328e-10,4.824629e-09,400,410,100
8,1.788090e-09,1.326262e-10,2.412315e-09,400,410,100
9,2.144303e-09,1.557213e-10,2.831479e-09,400,410,100


In [21]:
import pandas as pd
import numpy as np

from skopt.space import Real, Integer
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

In [22]:
# Define the true objective functions
def obj_func(X, Y):
    return  0.0001 * (np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**0.1

def obj_func2(X, Y):
    return -0.0001 * (np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**0.1 

In [23]:
ran_err = lambda n, x: np.random.normal(0, x, n)

In [24]:
# number of initial simulations
sim_n = 10

# Random Seed
r = np.random.default_rng(seed=2)

In [25]:
xy_range = [(-10, 10)]*2

X = np.arange(*xy_range[0], ((xy_range[0][1] - xy_range[0][0]) / 100))
Y = np.arange(*xy_range[1], ((xy_range[1][1] - xy_range[1][0]) / 100))
X, Y = np.meshgrid(X, Y)

# Generate the initial simulated experiments
df = pd.DataFrame(r.uniform(xy_range[0][0], xy_range[0][1], size=(sim_n, 2)))
df.columns = ["x", "y"]
df["z1"] = obj_func(df['x'], df['y']) + ran_err(sim_n, 0.05)
df["z2"] = obj_func2(df['x'], df['y']) + ran_err(sim_n, 0.05)

In [26]:
df

,x,y,z1,z2
0,-4.767757,-4.030177,1.725951,-1.751426
1,6.284515,-8.161681,0.762713,-0.798049
2,2.002011,4.571211,1.824039,-1.909504
3,-6.241979,-8.897067,1.090861,-1.107814
4,-4.500613,3.148660,1.115001,-1.139269
5,1.245313,-6.998755,1.615774,-1.645682
6,-1.347384,3.385946,1.669582,-1.739672
7,-1.544307,2.663688,1.833823,-1.883781
8,9.348719,3.661296,1.121857,-1.256210
9,-2.167503,-6.254949,1.274080,-1.129686


In [27]:
# Define the hyperparameter space
hyp_space  = [
    Integer(2, 10, name='num_layers'), # Num of layers in the network (depth)
    Integer(50, 200, name='num_units'), # Num of neurons in each hidden layer (width)
    Real(0.00001, 0.1, prior='log-uniform', name='learning_rate'), # Steps size at each iteration 
    Real(0.0, 0.5, name='dropout_rate'), # Probability of droping out a neuron
    Integer(20, 150, name='batch_size'), # Num of samples per batch
    Integer(50, 200, name='epochs') # Num of epochs (iterations over the entire dataset) during training
]

# Defining the Neural Network Model
def NN_model(num_layers, num_units, learning_rate, dropout_rate):
    inputs = Input(shape=(2,))
    x = Dense(num_units, activation='relu', kernel_regularizer='l2')(inputs)
    for _ in range(num_layers - 1):
        x = Dense(num_units, activation='relu', kernel_regularizer='l2')(x)
        x = Dropout(dropout_rate)(x)
    outputs = Dense(2, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Define the objective function to minimize
@use_named_args(hyp_space)
def objective(**params):
    num_layers = params['num_layers']
    num_units = params['num_units']
    learning_rate = params['learning_rate']
    dropout_rate = params['dropout_rate']
    batch_size = params['batch_size']
    epochs = params['epochs']

    model = NN_model(num_layers, num_units, learning_rate, dropout_rate)
    
    # Define K-fold cross-validation
    kfold = KFold(n_splits=3)
    scores = []

    # Perform cross-validation
    for train_idx, val_idx in kfold.split(df[['x', 'y']]):
        X_train, X_val = df[['x', 'y']].iloc[train_idx], df[['x', 'y']].iloc[val_idx]
        y_train, y_val = df[['z1', 'z2']].iloc[train_idx], df[['z1', 'z2']].iloc[val_idx]

        # Standardize the features
        scaler_x = StandardScaler().fit(X_train)
        X_train_scaled = scaler_x.transform(X_train)
        X_val_scaled = scaler_x.transform(X_val)

        # Train the model
        model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        # Evaluate the model
        score = model.evaluate(X_val_scaled, y_val, verbose=0)
        scores.append(score)
    
    return np.mean(scores)

In [28]:
# Perform Bayesian optimization
result = gp_minimize(objective, hyp_space, n_calls=50, random_state=0, acq_func='EI')

# Output best hyperparameters from BayesOpt
print("Best hyperparameters:")
print("num_layers:", result.x[0])
print("num_units:", result.x[1])
print("learning_rate:", result.x[2])
print("dropout_rate:", result.x[3])
print("batch_size:", result.x[4])
print("epochs:", result.x[5])

# Train the model with the best hyperparameters on the entire dataset
best_model = NN_model(
    num_layers=result.x[0],
    num_units=result.x[1],
    learning_rate=result.x[2],
    dropout_rate=result.x[3]
)

KeyboardInterrupt: 

In [ ]:
# Standardize the entire dataset
scaler_x = StandardScaler().fit(df[['x', 'y']])
X_scaled = scaler_x.transform(df[['x', 'y']])

# Train the best model on the full dataset
best_model.fit(X_scaled, df[['z1', 'z2']], epochs=result.x[5], batch_size=result.x[4], verbose=1)